#### ex 1. nltk text generation
Train an ngram model on the english bible and get it to generate random text. Experiment with different start words and window lengths.
How intelligible is the text?
Discuss the strengths and weaknesses of this method of generating random text and propose alternatives.

In [1]:
import nltk
nltk.download('gutenberg')
nltk.corpus.gutenberg.fileids()

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/valeriia/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [2]:
bible = nltk.corpus.gutenberg.words('bible-kjv.txt')
def generate_model(cfdist, word, num=20):
    for i in range(num):
        print(word),
        word = cfdist[word].max()
        
bigrams = nltk.bigrams(bible)
cfd = nltk.ConditionalFreqDist(bigrams)

generate_model(cfd, "never")

never
be
a
man
,
and
the
LORD
,
and
the
LORD
,
and
the
LORD
,
and
the
LORD


#### ex 2. wn meronyms
Use the BROWN religion vs. romance subcorpora.
Which one has more NOUN meronyms? Remember that there are two types of meronyms.

In [3]:
from nltk.corpus import brown
brown.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [5]:
words_rel = brown.words(categories="religion")
words_rom = brown.words(categories="romance")

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = nltk.WordNetLemmatizer()
from nltk.corpus import wordnet as wn

def meronyms_count(text):
    text_lemmatized = [lemmatizer.lemmatize(w) for w in text]
    counter = 0
    for w in set(text_lemmatized):
        for synset in wn.synsets(w, wn.NOUN):
            for meronym in wn.synset(synset.name()).part_meronyms() or wn.synset(synset.name()).substance_meronyms():
                if meronym.name()[:-5] in text_lemmatized:
                    counter += 1
    return len(set(text_lemmatized)), counter

print("Religion vocab:",meronyms_count(words_rel)[0],",", "meronyms:",meronyms_count(words_rel)[1],",","meronyms %:",meronyms_count(words_rel)[1]/meronyms_count(words_rel)[0])
print("Romance vocab:",meronyms_count(words_rom)[0],",", "meronyms:",meronyms_count(words_rom)[1],",","meronyms %:",meronyms_count(words_rom)[1]/meronyms_count(words_rom)[0])


Religion vocab: 5875 , meronyms: 358 , meronyms %: 0.060936170212765955
Romance vocab: 7916 , meronyms: 804 , meronyms %: 0.10156644770085901


#### ex 3. wn polysemy
a) The polysemy of a word is the number of senses it has. Using the BROWN religion vs. romance subcorpora, which subcorpus is generally more polysemous or ‘ambiguous’?
b) Compute the average polysemy for the religion NOUNS in a 2 liner!

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = nltk.WordNetLemmatizer()
from nltk.corpus import wordnet as wn

def ambiguity(text):
    text_lemmatized = [lemmatizer.lemmatize(w) for w in text]
    counter = 0
    for w in set(text_lemmatized):
        for synset in wn.synsets(w):
            counter += 1
    return counter/len(set(text_lemmatized))

from nltk.corpus import brown
words_rel = brown.words(categories="religion")
words_rom = brown.words(categories="romance")
print("Religion: avg. number of meanings per word:",ambiguity(words_rel))
print("Romance: avg. number of meanings per word:",ambiguity(words_rom))

Religion: avg. number of meanings per word: 5.287489361702128
Romance: avg. number of meanings per word: 5.0828701364325415


#### ex 4. wn word similarity
wn provides 6 different measures of word similarity.
Which of these measures predicts the human ratings from the ‘simlex999’ gold standard best?

In [7]:
import pandas as pd
simlex999 = pd.read_csv("SimLex-999.txt", sep="\t")
simlex999

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93
...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75


In [19]:
from nltk.corpus import wordnet as wn

from nltk.corpus import genesis
genesis_ic = wn.ic(genesis, False, 0.0)

import numpy as np
from scipy import stats

simlex_scores = []
wordnet_scores = []
for x, y, pos, simlex_score in zip(simlex999.word1, simlex999.word2, simlex999.POS, simlex999.SimLex999):
    w1 = wn.synset("{}.{}.01".format(x, pos.lower()))
    w2 = wn.synset("{}.{}.01".format(y, pos.lower()))
    try:
        path = w1.path_similarity(w2)
        lch = w1.lch_similarity(w2)
        wup = w1.wup_similarity(w2)
        res = w1.res_similarity(w2, genesis_ic)
        jcn = w1.jcn_similarity(w2, genesis_ic)
        lin = w1.lin_similarity(w2, genesis_ic)
        wn_scores = [path, lch, wup, res, jcn, lin]
        for s in wn_scores:
            if s == None:
                raise ValueError("None")
        simlex_scores.append(simlex_score)
        wordnet_scores.append(wn_scores)
    except:
        continue

simlex_scores = np.asarray(simlex_scores)  
wordnet_scores = np.asarray(wordnet_scores)
n = wordnet_scores.shape[1]
wordnet_measures = ["path", "lch", "wup", "res", "jcn", "lin"]
for i in range(n):
    print(wordnet_measures[i],stats.pearsonr(wordnet_scores[:,i],simlex_scores))

path (0.4388878679612576, 4.15584005221031e-43)
lch (0.4339421864617497, 4.461004522769442e-42)
wup (0.3505600823618936, 4.4910144954556744e-27)
res (0.11777705210549169, 0.00043648545526834523)
jcn (0.23758159166048723, 7.350903788703595e-13)
lin (0.2831489470556223, 7.805533206291413e-18)
